In [1]:
#1. scrape Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
#2. get the latitude and the longitude coordinates of each neighborhood
#3. explore and cluster the neighborhoods in Toronto

In [80]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [29]:
#scrape with pandas
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(wikiurl)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [30]:
#Ignore cells with a borough that is Not assigned
df=df[df["Borough"] != "Not assigned"]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [31]:
#More than one neighborhood can exist in one postal code area, join neighbourhood with comma
df = df.groupby(["Postal Code", "Borough"]).Neighbourhood.agg([("Neighbourhood", ', '.join)]).reset_index()
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [32]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df["Neighbourhood"] == 'Not assigned', "Neighbourhood"] = df["Borough"]

In [33]:
#shape of the dataframe
df.shape

(103, 3)

In [34]:
#read geospatial data
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
#join geosptial data with dataframe
df_merge = pd.merge(df, df_geo, how="left", on="Postal Code")
df_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [69]:
#select only rows with the word Toronto in Borough
df_toronto = df_merge[df_merge.Borough.str.contains('Toronto')]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [70]:
#import folium and geocoders Nominatim
import folium
from geopy.geocoders import Nominatim

In [71]:
#explore the neighbourhood

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [72]:
#import k-means library
from sklearn.cluster import KMeans

In [73]:
#k-means clustering of Toronto neighbourhood
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_toronto[["Latitude", "Longitude"]])

#check clustering labels
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [74]:
#add clustering labels to dataframe
df_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

In [76]:
#check dataframe with clustering labels
df_toronto.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,1,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,1,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [81]:
#create Toronto map wth clustering
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood'], df_toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters